In [34]:
import pandas as pd

# 読み込み先のパス
load_path = "/Volumes/public/Document/Python/VAE/train/latent_yhat2.csv"

# CSVファイルを読み込む
train = pd.read_csv(load_path, encoding='utf-8-sig')

# 読み込んだデータフレームを表示
print(train)

              ds  VAE_MinesystemBasic_z_yhat  VAE_ModeMass_z_yhat  \
0     2024-03-29                    0.530606             0.902087   
1     2024-03-31                    0.624744             0.990880   
2     2024-04-02                    0.672332             0.794808   
3     2024-04-05                    0.561498             0.892236   
4     2024-04-08                    0.489208             0.826903   
...          ...                         ...                  ...   
2238  2206-08-01                  305.714541           -92.822371   
2239  2206-09-01                  305.770562           -92.927109   
2240  2206-10-01                  306.006499           -92.791249   
2241  2206-11-01                  306.366391           -93.118353   
2242  2206-12-01                  306.187569           -93.055175   

      VAE_SportKorea_z_yhat  VEA_StreetNatural_z_yhat  \
0                  0.635001                  0.086204   
1                  0.421620                 -0.102709   


In [35]:
# 最初の行を取得
first_row = train.iloc[0]
print(first_row)

ds                            2024-03-29
VAE_MinesystemBasic_z_yhat      0.530606
VAE_ModeMass_z_yhat             0.902087
VAE_SportKorea_z_yhat           0.635001
VEA_StreetNatural_z_yhat        0.086204
VAE_OutdoorNormcore_z_yhat      1.095633
VAE_EleganceTrad_z_yhat         0.249244
VAE_GirlyDesigner_z_yhat        1.114815
VAE_OfficeManush_z_yhat         1.959909
Name: 0, dtype: object


In [36]:
beas_pass = "/Volumes/public/Document/Python/VAE/data3/models/"

vae_models = [
    "vae_model.pth",
    "VAE_ModeMass.pth",
    "VAE_SportKorea.pth",
    "VEA_StreetNatural.pth",
    "VAE_OutdoorNormcore.pth",
    "VAE_EleganceTrad.pth",
    "VAE_GirlyDesigner.pth",
    "VAE_OfficeManush.pth"
]

model_paths = []
for i in range(len(vae_models)):
    path = beas_pass+vae_models[i]
    model_paths.append(path)

model_paths

['/Volumes/public/Document/Python/VAE/data3/models/vae_model.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VAE_ModeMass.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VAE_SportKorea.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VEA_StreetNatural.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VAE_OutdoorNormcore.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VAE_EleganceTrad.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VAE_GirlyDesigner.pth',
 '/Volumes/public/Document/Python/VAE/data3/models/VAE_OfficeManush.pth']

In [37]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image, UnidentifiedImageError
import os
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# モデルの定義
class Encoder(nn.Module):
    def __init__(self, latent_size, image_size):
        super(Encoder, self).__init__()
        self.H = int(image_size / 4)  # ダウンサンプリングされるサイズ
        self.W = int(image_size / 4)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.linear1 = nn.Linear(128 * self.H * self.W, 64)
        self.linear2 = nn.Linear(64, latent_size)  # 潜在変数の次元を1に変更
        self.linear3 = nn.Linear(64, latent_size)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.shape[0], -1)  # Flatten
        x = F.relu(self.linear1(x))
        z_mean = self.linear2(x)
        z_log_var = self.linear3(x)
        return z_mean, z_log_var

    def sampling(self, z_mean, z_log_var):
        epsilon = torch.randn(z_mean.shape, device=z_mean.device)
        return z_mean + epsilon * torch.exp(0.5 * z_log_var)

class Decoder(nn.Module):
    def __init__(self, image_size):
        super().__init__()
        self.H = int(image_size / 4)
        self.W = int(image_size / 4)
        self.to_shape = (128, self.H, self.W)
        self.linear = nn.Linear(latent_size, 128 * self.H * self.W)  # 潜在サイズを1に変更
        self.deconv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x = F.relu(self.linear(x))
        x = x.view(-1, *self.to_shape)  # Reshape to (batch, C, H, W)
        x = F.relu(self.deconv1(x))
        x = torch.sigmoid(self.deconv2(x))
        return x

class VAE(nn.Module):
    def __init__(self, latent_size, image_size):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_size, image_size)
        self.decoder = Decoder(image_size)

    def forward(self, x, C=1.0, k=1):
        z_mean, z_log_var = self.encoder(x)

        rec_loss = 0
        for _ in range(k):
            z = self.encoder.sampling(z_mean, z_log_var)
            y = self.decoder(z)
            rec_loss += F.binary_cross_entropy(torch.flatten(y, start_dim=1), torch.flatten(x, start_dim=1)) / k

        kl_loss = C * (z_mean ** 2 + torch.exp(z_log_var) - z_log_var - 1) * 0.5
        kl_loss = torch.sum(kl_loss) / len(x)
        return rec_loss + kl_loss

In [ ]:
import torch
import matplotlib.pyplot as plt
import os

# 各モデルのロード
models = []
latent_size = 1  # 各モデルの潜在変数の次元
image_size = 276  # 画像サイズ
device = torch.device("cpu")

for i in range(8):
    model = VAE(latent_size=latent_size, image_size=image_size).to(device)
    model.load_state_dict(torch.load(model_paths[i], map_location=device))
    model.eval()
    models.append(model)

def generate_combined_image(models, z_values, filename):
    # z_valuesをTensorに変換
    z_values = torch.tensor(z_values, dtype=torch.float32).to(device)

    # 各モデルのデコーダを使って画像を生成
    with torch.no_grad():
        generated_images = []
        for i in range(len(models)):
            img = models[i].decoder(z_values[i].view(1, -1)).cpu().numpy()
            generated_images.append(img)

    # 画像を平均して結合
    combined_img = np.mean(generated_images, axis=0)

    # 画像の形状を (276, 276, 3) に変換
    combined_img = combined_img[0].transpose(1, 2, 0)

    # 生成された画像を表示
    print(z_values.cpu().numpy())
    plt.imshow(combined_img)
    plt.title(f'{filename}')
    plt.axis('off')
    plt.show()

    # 画像を保存
    plt.savefig('/Volumes/public/Document/Python/VAE/predict_img/'+filename, bbox_inches='tight', pad_inches=0)
    plt.close()  # プロットを閉じる

for i in range(len(train)):
    # 最初の行を取得
    first_row = train.iloc[i]
    z_values = first_row[1:9]
    z_values = list(z_values)

    generate_combined_image(models, z_values, first_row[0])

/var/folders/1f/9sw91vb56l9dpgy8zb1m887w0000gn/T/ipykernel_13342/4048708109.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_paths

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os

# 各モデルのロード
models = []
latent_size = 1  # 各モデルの潜在変数の次元
image_size = 276  # 画像サイズ
device = torch.device("cpu")

# モデルが保存されているディレクトリのパス
model_directory = "/Volumes/public/Document/Python/VAE/data3/models"

# モデルファイルのパスを取得
model_paths = [os.path.join(model_directory, filename) for filename in os.listdir(model_directory) if os.path.isfile(os.path.join(model_directory, filename))]

for i in range(8):
    model = VAE(latent_size=latent_size, image_size=image_size).to(device)
    model.load_state_dict(torch.load(model_paths[i], map_location=device))
    model.eval()
    models.append(model)

def generate_combined_image(models, z_values, filename):
    # z_valuesをTensorに変換
    z_values = torch.tensor(z_values, dtype=torch.float32).to(device)

    # 各モデルのデコーダを使って画像を生成
    with torch.no_grad():
        generated_images = []
        for i in range(len(models)):
            img = models[i].decoder(z_values[i].view(1, -1)).cpu().numpy()
            generated_images.append(img)

    # 画像を平均して結合
    combined_img = np.mean(generated_images, axis=0)

    # 画像の形状を (276, 276, 3) に変換
    combined_img = combined_img[0].transpose(1, 2, 0)

    # 画像の値を0〜1にスケーリング
    combined_img = (combined_img - combined_img.min()) / (combined_img.max() - combined_img.min())

    # 保存先のディレクトリを設定
    save_directory = '/Volumes/public/Document/Python/VAE/predict_img'
    os.makedirs(save_directory, exist_ok=True)
    save_path = os.path.join(save_directory, f'{filename}.png')

    # 画像を保存
    plt.imsave(save_path, combined_img)
    print(f"画像を保存しました: {save_path}")

# データフレームの各行について画像を生成
for i in range(len(train)):
    # 最初の行を取得
    first_row = train.iloc[i]
    z_values = first_row[1:9].values.tolist()  # z_valuesをリストに変換
    filename = str(first_row[0])  # ファイル名用の値を取得

    generate_combined_image(models, z_values, filename)

/var/folders/1f/9sw91vb56l9dpgy8zb1m887w0000gn/T/ipykernel_13342/61821325.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_paths[i

画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-03-29.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-03-31.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-02.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-05.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-08.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-15.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-16.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-18.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-19.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-22.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-24.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-04-26.png
画像を保存しました: /Volumes/public/Document/Python/VAE/predict_img/2024-05-06.png
画像を保存しました: /Volumes/public/Document/Py

KeyboardInterrupt: 